##### Let's take the CommaSeparatedListOutputParser example from Day 3 and rebuild it using LCEL.

### --- Understanding the Flow (Conceptual) ---
1. chain.invoke({"topic": "..."}) is called.
2. The input dictionary {"topic": "..."} is passed to `prompt_template`.
3. `prompt_template` formats the prompt string using the topic and format_instructions.
4. The formatted prompt (which is a list of messages for ChatPromptTemplate) is passed to `llm`.
5. `llm.invoke()` processes the prompt and returns an AIMessage.
6. The AIMessage object is passed to `list_parser`.
7. `list_parser.parse()` extracts the `.content` from the AIMessage and parses it.
8. The final parsed list is returned by `chain.invoke()`.

In [ ]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import CommaSeparatedListOutputParser

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")


In [ ]:
#1. Output Parser
list_parser = CommaSeparatedListOutputParser()
format_instruction = list_parser.get_format_instructions()

#2. prompt template
prompt_template = ChatPromptTemplate.from_template(
    "List 5 specific example related to topic : {topic}.\n {format_instruction}"
)

#3. LLM configuration
llm = ChatOpenAI(
    openai_api_key=api_key,
    model="gpt-3.5-turbo",
    temperature=0.8,
    max_tokens=1000,
)

#4. Partial prompt: prefill the format_instruction variable. The resulting partial prompt object only expect the remaining
# variable to be filled in i.e. topic
partial_prompt = prompt_template.partial(format_instruction=format_instruction)

In [ ]:
# Build the chain using LCEL
# The | operetor is used to chain the components together
# The flow is: prompt template -> LLM -> output parser
chain = partial_prompt | llm | list_parser

print("LCEL Chain")
print(chain) 


In [ ]:
# Invoke the chain
input_data = {"topic": "popular ai agent frameworks"}

try:
    result = chain.invoke(input_data)
    print(f"\nChain output (passed through parser): {result}")
    print(f"Type of result: {type(result)}")

    # Lets try another topic
    input_data_2 = {"topic": "popular online AIML course which has a good rating"}
    result_2 = chain.invoke(input_data_2)
    print(f"\nChain output2 (passed through parser): {result_2}")

except Exception as e:
    print(f"Error invoking chain: {e}")
    print(f"Type of error: {type(e)}")
    print(f"Error message: {str(e)}")
    if hasattr(e, 'response'):
        print(f"Response: {e.response}")
    if hasattr(e, 'status_code'):
        print(f"Status code: {e.status_code}")
    if hasattr(e, 'headers'):
        print(f"Headers: {e.headers}")